In [29]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from config import api_key
%matplotlib inline

In [30]:
quandl.ApiConfig.api_key = api_key
#Variance Risk Premium: https://sites.google.com/site/haozhouspersonalhomepage
#EOM Nominal S&P, Nominal Dividends / Nominal Risk Free Rate: http://www.hec.unil.ch/agoyal/
#Consumer Sentiment: http://www.sca.isr.umich.edu/

In [45]:
ticker = 'SPY'
transform = 'normalize'
start_date = '2004-12-31'
end_date = '2018-01-01'

In [46]:
index_data = quandl.get('EOD/' + ticker, start_date=start_date, end_date=end_date, column_index=4, collapse='monthly')

In [47]:
risk_free_rate = quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date, collapse='monthly')

In [48]:
index_data = index_data.reset_index()
risk_free_rate = risk_free_rate.reset_index()

In [50]:
index_data.tail()
risk_free_rate.tail()

,Date,Close
152,2017-08-31,247.49
153,2017-09-30,251.23
154,2017-10-31,257.15
155,2017-11-30,265.01
156,2017-12-31,266.86


,Date,Value
152,2017-08-31,0.99
153,2017-09-30,1.04
154,2017-10-31,1.13
155,2017-11-30,1.25
156,2017-12-31,1.37


In [51]:
#Calculate Monthly Risk Free Rate
risk_free_list = []

for i in range(len(risk_free_rate)):
    x = (risk_free_rate.iloc[i]['Value']/12)/100
    risk_free_list.append(x)

In [52]:
#Concat to Risk Free DF
risk_free_rate = pd.concat([risk_free_rate, pd.Series(data=risk_free_list, name='risk_free_rate')], axis=1)

In [53]:
risk_free_rate.tail(10)

,Date,Value,risk_free_rate
147,2017-03-31,0.75,0.000625
148,2017-04-30,0.79,0.000658
149,2017-05-31,0.96,0.000800
150,2017-06-30,1.01,0.000842
151,2017-07-31,1.05,0.000875
152,2017-08-31,0.99,0.000825
153,2017-09-30,1.04,0.000867
154,2017-10-31,1.13,0.000942
155,2017-11-30,1.25,0.001042
156,2017-12-31,1.37,0.001142


In [66]:
"""
Monthly >>>
Index: S&P 500 Index Data
D12: Rolling Cumulative Dividends Over Past 12 Months 
Rfree: Nominal Risk Free Rate
"""
goyal = pd.read_excel('PredictorData2017.xlsx')
df_goyal = pd.DataFrame(goyal)
df_goyal.head(10)

'\nMonthly >>>\nIndex: S&P 500 Index Data\nD12: Rolling Cumulative Dividends Over Past 12 Months \nRfree: Nominal Risk Free Rate\n'

,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
0,187101,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004955,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,187102,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004514,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,187103,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004243,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,187104,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004632,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,187105,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003691,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,187106,4.82,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003368,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,187107,4.73,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,187108,4.79,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003706,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,187109,4.84,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004544,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,187110,4.59,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.007016,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df_cols = ['yyyymm', 'Index', 'D12', 'Rfree']
df = df_goyal[df_cols]
df.head()

,yyyymm,Index,D12,Rfree
0,187101,4.44,0.26,0.004955
1,187102,4.50,0.26,0.004514
2,187103,4.61,0.26,0.004243
3,187104,4.74,0.26,0.004632
4,187105,4.86,0.26,0.003691


In [57]:
"""
VRP: Variance Risk Premium
"""
vrp = pd.read_fwf('VRPtable.txt')
df_vrp = pd.DataFrame(vrp)
df_vrp.head(10)

'\nVRP: Variance Risk Premium\n'

,Year,Month,VRP,EVRP,IV,RV,ERV
0,1990,1,35.9054,35.9054,53.5941,17.6888,17.6888
1,1990,2,32.9177,15.4492,40.2967,7.3790,24.8475
2,1990,3,26.5978,16.0745,32.4394,5.8416,16.3649
3,1990,4,26.2753,18.6211,31.7525,5.4773,13.1314
4,1990,5,19.2201,12.3841,25.1431,5.9229,12.7589
5,1990,6,13.0796,9.3027,20.0208,6.9412,10.7181
6,1990,7,26.9870,27.7368,37.1360,10.1490,9.3993
7,1990,8,51.0203,58.1133,74.5008,23.4805,16.3875
8,1990,9,57.6982,36.5006,70.6160,12.9178,34.1155
9,1990,10,52.1602,46.5117,75.2001,23.0400,28.6884


In [65]:
vrp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 7 columns):
Year     348 non-null int64
Month    348 non-null int64
VRP      348 non-null float64
EVRP     348 non-null float64
IV       348 non-null float64
RV       348 non-null float64
ERV      348 non-null float64
dtypes: float64(5), int64(2)
memory usage: 19.1 KB
